In [24]:
import pandas as pd
from tqdm import tqdm
import re
import os
import random
import nltk
import collections

random.seed(1991)

In [3]:
candor_path = "/home/prevot/data/no_media/"
dirs = os.listdir(candor_path)
len(dirs)

cliffhanger = pd.read_csv(os.path.join(candor_path, dirs[0], "transcription/transcript_cliffhanger.csv"))


1656

In [16]:
cand_utts = []
for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    utts = cliffhanger['utterance'].tolist()
    utts = [re.sub(r"(\w)([^\w\s'])", r"\1 #", x) for x in utts]
    utts = [x.split() for x in utts if 'uh huh' not in x.lower() and 'uh mhm' not in x.lower()]
    utts = [x for x in utts if len(x) > 4 and len(x) < 31 and ('uh' in x or 'Uh' in x or 'um' in x or 'Um' in x)]
    cand_utts = cand_utts + utts

inis = [u[0] for u in cand_utts if len(u)>4]
collections.Counter(inis)

In [52]:
DMS = ['so','but',"because","well",'like']
mono_canditates = []
cnt = 0
cnt2 = 0
cnt3 = 0

for i in dirs:
    cnt+=1
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    for item,row in cliffhanger.iterrows():
        cnt2 +=1 
        utts = row['utterance'].lower().split('.')
        #print(utts)
        #input('???')
        if len(utts)>1:
            pairs = nltk.bigrams(utts)
            for pair in pairs:
                cnt3 +=1
                if (20 > len(pair[0].split())>3) and (20 >len(pair[1].split())>3):
                    if pair[1].split()[0] in DMS:
                        mono_canditates.append(pair[0]+'<$>'+pair[1])

print(cnt)
print(cnt2)
print(cnt3)
print(len(mono_canditates))

1656
530316
974368
14054


In [53]:
random.shuffle(mono_canditates)
mono_canditates = mono_canditates[0:500]
foutput = open('data_corpus/english_DM_mono.txt','w')
towrite_lines = '\n'.join(mono_canditates)
foutput.writelines(towrite_lines)
foutput.close()

In [50]:
DMS = ['so','but',"because","well",'like']
dial_canditates = []
dial_dm_stats = []

for i in dirs:
    cliffhanger = pd.read_csv(os.path.join(candor_path, i, "transcription/transcript_cliffhanger.csv"))
    cliffhanger['next_utt'] = cliffhanger['utterance'].shift(-1)
    cliffhanger['next_id'] = cliffhanger['turn_id'].shift(-1)
    
    for i,row in cliffhanger.iterrows():
        if (20 > len(row['utterance'].split()) > 3) and (20 > len(str(row['next_utt']).split()) > 3) and (row['next_id']!=0):
            first_tok = row['next_utt'].split()[0]
            if first_tok.lower() in DMS: 
                dial_canditates.append(row['utterance'].lower()+'<$>'+row['next_utt'].lower())
                dial_dm_stats.append(first_tok)

print(collections.Counter(dial_dm_stats))
print(len(dial_canditates))

Counter({'So': 4133, 'But': 1852, 'Like': 896, 'Well': 856, 'Because': 97, 'but': 5, 'so': 5, 'well': 2, 'like': 1})
7847


In [51]:
random.shuffle(dial_canditates)
dial_canditates = dial_canditates[0:500]
foutput = open('data_corpus/english_DM_dial.txt','w')
towrite_lines = '\n'.join(dial_canditates)
foutput.writelines(towrite_lines)
foutput.close()         
                    
